<a href="https://colab.research.google.com/github/torontoniancoder/python_tutorial/blob/master/ML_with_linear_regression_in_Python_final_project_C05_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [237]:
import pandas as pd
import numpy as np

A. Data cleaning

In [238]:
def clean_data(df):
    pd.set_option('mode.use_inf_as_na', True) # to recognise inf as a NaN in calculations: from here: https://stackoverflow.com/questions/58225701/change-pandas-default-fillna-setting
    # todo: replace the nan values with relevant (??) values (MEAN?) -> replaced with 0-es
    # selecting the columns for conversion from categorical data into columns of binary data
    df_temp = df[['Product_ID','Gender','Age','Occupation', 'City_Category','Stay_In_Current_City_Years']]
    # integer encoding
    # source for instructions: https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40
    # turning the type columns into the dummy variables
    # No nan values therefore dummy_na=False
    dummy = pd.get_dummies(df_temp, dummy_na=False, columns=['Product_ID','Gender','Age','Occupation', 'City_Category','Stay_In_Current_City_Years'], prefix=['Product_ID','Gender','Age','Occupation', 'City_Category','Stay_In_Current_City_Years'], dtype=int, drop_first=False)
    # concatenating binary data into the initial dataframe
    df = pd.concat([df,dummy], axis=1)
    # dropping the unnecessary fields
    # FieldDrop0 = ['User_ID','Product_ID','Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years','Purchase']
    # df = df.drop(FieldDrop0, axis=1)
    colnames = df.columns.values.tolist() #converting column names into a list
    # colnames
    # counting nan values and removing columns with nan values more than 50% (threshold can be modified *there -> <-*)
    for each in colnames:
        # print(type(df[each]))
        if (( df[each].isna().sum() / len(df[each]) ) > 0.5): # *-> <-* here! the threshold can be modified | source URL: https://stackoverflow.com/questions/26266362/how-to-count-the-nan-values-in-a-column-in-pandas-dataframe
            print ('Due to low data quality\n(data not available for more than 50% of data points)\nI am deleting this column: '+each)
            # *-> <-* here! in the line above the threshold must be modified if it si changed in the previous line
            del df[each]
        else:
            df[each] = df[each].fillna(0)
            df[each] = df[each].astype('string')
            pass
            # z = ( df[each].isna().sum() / len(df[each]) )*100
            # print('For column '+str(each)+' unknown datapoints are '+str(z)+'% of series')
    # df.info()
    # df = df.astype('string') # source URL: https://stackoverflow.com/questions/22005911/convert-columns-to-string-in-pandas
    # df[["D", "E"]] = df[["D", "E"]].astype(int)
    #   cleaning temporary data
    df_temp = 0
    return df

In [239]:
df = pd.read_csv('Black_Friday.csv') #loading data into a dataframe

In [240]:
# *** REMOVE BEFORE ASSEMBLY *** BEGIN
df = df.iloc[:7000] # slicing dataframe for test&dev performance
# df.shape
# *** REMOVE BEFORE ASSEMBLY *** END


In [241]:
# keeping a control copy of the source dataset in variable df0
df0 = df.copy()

In [242]:
df = clean_data(df) # running function clean_data on a dataframe

Due to low data quality
(data not available for more than 50% of data points)
I am deleting this column: Product_Category_3


In [243]:
# df.info()
# display(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 1797 entries, User_ID to Stay_In_Current_City_Years_4+
dtypes: string(1797)
memory usage: 68.6 MB


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase,Product_ID_P00000142,Product_ID_P00000342,Product_ID_P00000442,Product_ID_P00000542,Product_ID_P00000642,Product_ID_P00001042,Product_ID_P00001142,Product_ID_P00001242,Product_ID_P00001542,Product_ID_P00001642,Product_ID_P00001742,Product_ID_P00001942,Product_ID_P00002042,Product_ID_P00002142,Product_ID_P00002242,Product_ID_P00002342,Product_ID_P00002442,Product_ID_P00002542,Product_ID_P00002642,Product_ID_P00002942,Product_ID_P00003242,Product_ID_P00003442,Product_ID_P00003642,Product_ID_P00003942,Product_ID_P00004242,Product_ID_P00004342,Product_ID_P00004442,Product_ID_P00004542,Product_ID_P00004642,...,Product_ID_P0099442,Product_ID_P0099742,Gender_F,Gender_M,Age_0-17,Age_18-25,Age_26-35,Age_36-45,Age_46-50,Age_51-55,Age_55+,Occupation_0,Occupation_1,Occupation_2,Occupation_3,Occupation_4,Occupation_5,Occupation_6,Occupation_7,Occupation_8,Occupation_9,Occupation_10,Occupation_11,Occupation_12,Occupation_13,Occupation_14,Occupation_15,Occupation_16,Occupation_17,Occupation_18,Occupation_19,Occupation_20,City_Category_A,City_Category_B,City_Category_C,Stay_In_Current_City_Years_0,Stay_In_Current_City_Years_1,Stay_In_Current_City_Years_2,Stay_In_Current_City_Years_3,Stay_In_Current_City_Years_4+
count,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,...,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000
unique,726,1748,2,7,21,3,5,2,18,18,3670,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,1000424,P00112142,M,26-35,4,B,1,0,5,0.0,7187,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
freq,52,19,3859,1947,655,1976,1654,3116,1439,1597,7,4991,4999,4999,4998,4990,4996,4997,4998,4999,4995,4996,4999,4998,4995,4997,4999,4997,4994,4999,4997,4993,4991,4991,4994,4997,4999,4999,4998,4999,...,4997,4995,3859,3859,4891,3840,3053,4089,4586,4691,4850,4434,4675,4801,4770,4345,4931,4838,4490,4953,4918,4890,4859,4589,4906,4812,4928,4809,4524,4924,4913,4691,3530,3024,3446,4227,3346,4167,4049,4211


C. Data Modeling

In [244]:
# source code URL: https://habr.com/ru/post/206306/
# and here, URL: https://habr.com/ru/post/202090/

# importing libraries

from pandas import read_csv, DataFrame
# from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
# from sklearn.svm import SVR
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import r2_score
# from sklearn import cross_validation, svm
# from sklearn.cross_validation import train_test_split

In [245]:
# creating dataset with summary data per UserID

# df_c1 = df.copy() #loading data into a dataframe

In [246]:
# df_c1 = clean_data(df_c1) #
# PreFinal_df = df.groupby('User_ID').sum()

In [247]:
# df_c1.info()

In [248]:
# df_c1 = df_c1.astype(str)

In [249]:
# df_c1.info()

In [250]:
# To begin with, we need to divide our training sample
# into the indicator that we are examining, and its defining features:
# todo: check features for cross-correlation, see source code
#  here, URL: http://espressocode.top/python-pandas-dataframe-corr/

# PreFinal_df

In [251]:
FieldDrop_PID = ['User_ID','Product_ID','Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years']
df = df.drop(FieldDrop_PID, axis=1)

In [252]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 1790 entries, Marital_Status to Stay_In_Current_City_Years_4+
dtypes: string(1790)
memory usage: 68.3 MB


In [253]:
df = df.astype(float)
# df.shape()
# df.info()

In [254]:
CorrKoeff = df.corr()

In [255]:
# CorrKoeff

In [256]:
colnames = 0

In [257]:
colnames = CorrKoeff.columns.values.tolist()

In [258]:
Feature_Drop = list(())
Feature_DropSoft = list(())

In [259]:
# checking the correlation between the features
for each in colnames:
    i = len(CorrKoeff[each])
    for i in range(i):
        if (CorrKoeff.iloc[i][each]  > 0.95) and (CorrKoeff.index[i] != each):
            # print('\n* *XX*|||*XX* * !!WARNING!! ['+str(each)+'] IS CORRELATED WITH '+str(CorrKoeff.index[i])+' BY '+str(CorrKoeff.iloc[i][each])+' * *XX*|||*XX* *')
            Feature_Drop.append(each)
            pass
        elif (CorrKoeff.iloc[i][each]  < -0.95) and (CorrKoeff.index[i] != each):
            # print('\n- *XX*XX* - Attention: ['+str(each)+'] is correlated with ['+str(CorrKoeff.index[i])+'] by '+str(CorrKoeff.iloc[i][each])+' - *XX*XX* -\n')
            Feature_DropSoft.append(each)
            pass
        else:
            pass
    # print('----- * ----- '+str(each)+' is [COMPLETE] ----- * -----')

In [260]:
# there is a correlation > 95% and <-95% between the features in df

In [261]:
# todo: drop the unnecessary features (columns)

In [262]:
# writing the Purchase data (targeted for prediction) into a separate dataset
target = df0['Purchase']
# dropping the unnecessary features


In [263]:
# FieldDrop1 = ['Purchase']

In [264]:
# df = df.drop(FieldDrop1, axis=1)

In [265]:
# df

In [266]:
len(Feature_Drop) # these are features that are absolutely certain to be removed

0

In [267]:
len(Feature_DropSoft) # these are features that are not certain to be removed

2

In [268]:
# Feature_DropSoft = Feature_DropSoft + Feature_Drop
# len(Feature_DropSoft)

In [269]:
UFeatures_Drop = list(())
UFeatures_DropSoft = list(())

In [270]:
[UFeatures_Drop.append(x) for x in Feature_Drop if x not in UFeatures_Drop]
[UFeatures_DropSoft.append(z) for z in Feature_DropSoft if z not in UFeatures_DropSoft]

[None, None]

In [271]:
length = len(UFeatures_Drop)
lengthSoft = len(UFeatures_DropSoft)

In [272]:
middle_index = length//2 # source code URL: https://www.kite.com/python/answers/how-to-split-a-list-in-half-in-python
middle_indexSoft = lengthSoft//2

In [273]:
UFeatures_Droph = UFeatures_Drop[:middle_index]
UFeatures_DropSofth = UFeatures_DropSoft[:middle_index]

In [274]:
UFeatures_Drop_United = UFeatures_Droph + UFeatures_DropSofth

In [275]:
len(UFeatures_Drop_United)

0

In [276]:
# df

In [277]:
newdf = df.drop(UFeatures_Drop_United, axis=1).copy()

In [278]:
# newdf

In [279]:
# newdf2 = newdf.drop(Feature_DropSoft, axis=1).copy()

In [280]:
# newdf2.shape()

In [281]:
# newtr1 = newtr[['Gender_F','Gender_M','Age_0-17','Age_18-25','Age_26-35','Age_36-45','Age_46-50','Age_51-55','Age_55+','City_Category_A','City_Category_B','City_Category_C','Stay_In_Current_City_Years_0','Stay_In_Current_City_Years_1','Stay_In_Current_City_Years_2','Stay_In_Current_City_Years_3','Stay_In_Current_City_Years_4+']].copy()

In [282]:
# models = [LinearRegression(), # метод наименьших квадратов
#	          RandomForestRegressor(n_estimators=100, max_features ='sqrt'), # случайный лес
#	          KNeighborsRegressor(n_neighbors=6), # метод ближайших соседей
#	          SVR(kernel='linear'), # метод опорных векторов с линейным ядром
#	          LogisticRegression() # логистическая регрессия
#	          ]



In [283]:
from sklearn.model_selection import train_test_split

In [284]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(newdf, target, test_size=0.2)


In [285]:
print (Xtrn.shape)
print (Ytrn.shape)
print (Xtest.shape)
print (Ytest.shape)
# print X_test.shape, y_test.shape
# source code URL: https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

(4000, 1790)
(4000,)
(1000, 1790)
(1000,)


In [286]:
# fit a model
lm = LinearRegression()
model = lm.fit(Xtrn, Ytrn)
predictions = lm.predict(Xtest)


In [287]:
# Let’s see what (some of) the predictions are:
predictions[0:5]

array([ 6078.        ,  6182.        , 15275.00000576,  6121.99998309,
        7037.        ])

In [288]:
Ytest = Ytest.astype(float)
predictions = predictions.astype(float)

In [289]:
# source code URL: https://python-school.ru/linear-regression-basis/

# Ytest = np.exp(Ytest)
# predictions = np.exp(predictions)

In [290]:
# source code URL: https://python-school.ru/linear-regression-basis/
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
mse = mean_squared_error(Ytest, predictions)
mae = mean_absolute_error(Ytest, predictions)
print('Mean Squared Error: %.3f, Mean Squared Error: %.3f' % (mse, mae))

Mean Squared Error: 0.000, Mean Squared Error: 0.000


In [291]:
# import seaborn as sns

In [292]:
# sns.pairplot(newdf)


In [293]:
# features = newdf.columns.values.tolist()

In [294]:
# sns.pairplot(newdf, x_vars=[features], y_vars='predictions', kind='reg', height=100)